In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import AllChem as ch
from rdkit.Chem import Draw as d
import pandas as pd
from rdkit import DataStructs
import collections
from rdkit.Chem.MolStandardize import rdMolStandardize
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('../0 data preparation/All.csv')

In [3]:
def Murcko_Skeleton(df,filename):
    # largest molecular fragment
    Standard_Smiles=df.Standard_Smiles.values
    mols=[]
    for i in Standard_Smiles:
        mol=Chem.MolFromSmiles(i)
        mol_frags = Chem.rdmolops.GetMolFrags(mol, asMols = True)
        largest_mol = max(mol_frags, default=mol, key=lambda m: m.GetNumAtoms())
        mols.append(largest_mol)
    # Atomic scaffold of all molecules in the database
    database_atomic_scaffolds = [MurckoScaffold.GetScaffoldForMol(mol) for mol in mols]
    for i in database_atomic_scaffolds:
        i.Compute2DCoords()
    # graph scaffold of all molecules in the database
    def genericize_scaffold(s):
        try:
            return MurckoScaffold.MakeScaffoldGeneric(s)
        except ValueError:
            return None
    database_grafh_scaffolds = [genericize_scaffold(s) for s in database_atomic_scaffolds]
    
    scaffold_smiles = [Chem.MolToSmiles(scaffold) for scaffold in database_atomic_scaffolds if scaffold != None]
    df['Scaffold_Smiles']=scaffold_smiles
    df.to_csv(filename+'_Scaffold.csv',index=False)
    # dominant scaffold
    counter=collections.Counter(scaffold_smiles)
    # save file
    columns = ['Smiles','Num']
    results = pd.DataFrame(counter.items(),columns=columns)
    results = results.sort_values(by=['Num'],ascending=False)
    results.reset_index(drop=True,inplace=True)
    results.to_csv(filename+'_Murcko.csv',index=False)

### The murcko scaffold of terpenoids in each content range

In [4]:
content_list=[0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10]
Content_Counts=[]
for i in range(len(content_list)):
    if i<=6:
        left=content_list[i]
        right=content_list[i+1]
        compound=df[(df['Content(%)']>=left)&(df['Content(%)']<right)]
        compound_dropduplicates=compound.drop_duplicates(subset='ID')
        compound_dropduplicates.to_csv(str(i+1)+'.csv',index=False)
        Murcko_Skeleton(compound_dropduplicates,str(i+1))